In [1]:
from glob import glob
import numpy as np
data_path = "/Users/soltan/Programs/kaggle/raw_data/train/train/*"

import librosa
def extract_features(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)

    return np.concatenate([mfccs, chroma, mel, contrast, tonnetz])


In [ ]:
from tqdm import tqdm
data_dir = np.array(glob(data_path))
features, labels = [], []
for file in tqdm(data_dir):
    file_name = file.split("/")[-1]
    file_name = file.split(".")[0]
    name, label = file_name.split("-")[0], file_name.split("-")[1]
    features.append(extract_features(file))
    labels.append(label)


  4%|▎         | 324/9000 [00:47<22:03,  6.56it/s]

In [ ]:
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, targets_train, targets_test = train_test_split(features, labels, test_size=0.2)


#
# from sklearn.neighbors import KNeighborsClassifier
#
# model = KNeighborsClassifier(n_neighbors=17)
#model.fit(inputs_train, targets_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

model = GridSearchCV(KNeighborsClassifier(), grid_params, n_jobs=10, verbose=1000)
model.fit(inputs_train, targets_train)

In [ ]:
from sklearn import metrics
predictions = model.predict(inputs_test)
print("Accuracy:",metrics.accuracy_score(targets_test, predictions))
